<a href="https://colab.research.google.com/github/faizankshaikh/ForaGym/blob/main/examples/trial1_foragymSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --quiet https://github.com/faizankshaikh/ForaGym.git
%cd ForaGym
!pip install -q -e .

In [1]:
import gym

import foragym

In [2]:
env = gym.make("foragym:foragym/ForaGym-v0")

In [4]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    done = False
    state = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not done:
        action = 0  # env.action_space.sample()
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, done, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Alive?: {not done}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [1 0 0 1 0]
--Current life: 4
--Type of Weather: Clear

Action to take: Forage
Chance to find food: 0.37
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 5
--State of Field: [1 1 1 1 0]
--Current life: 5
--Type of Weather: Clear

Action to take: Forage
Chance to find food: 0.95
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 4
--State of Field: [1 1 1 0 0]
--Current life: 6
--Type of Weather: Rainy

Action to take: Forage
Chance to find food: 0.73
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 3
--State of Field: [1 0 0 1 1]
--Current life: 6
--Type of Weather: Clear

Action to take: Forage
Chance to find food: 0.60
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 2
--State of Field: [1 1 1 1 0]
--Current life: 6
--Type of Weather: Rainy

Action to take: Forage
Chance to find food: 0.16
Reward?: -2
is Alive?: True

Current game state:-
--Days left: 1
--State of Field: [0 1

In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
model = DQN('MultiInputPolicy', env, verbose=1).learn(total_timesteps=int(2e5))